# 1. Import the necessary libraries if you are starting a new notebook. Using the same data as the previous lab: we_fn_use_c_marketing_customer_value_analysis.csv



In [124]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')

In [4]:
columns2 = []
for col in data.columns:
    columns2.append(re.sub(" ","_", col).lower())
data.columns = columns2
data.columns

Index(['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size'],
      dtype='object')

# 2. Find all of the categorical data. Save it in a categorical_df variable.



In [9]:
categorical_df = data.select_dtypes(object)
categorical_df.head()

,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


In [11]:
categorical_df = categorical_df.drop(columns = ['customer','effective_to_date'])

# 3. Check for NaN values.



In [12]:
categorical_df.isna().sum()

state               0
response            0
coverage            0
education           0
employmentstatus    0
gender              0
location_code       0
marital_status      0
policy_type         0
policy              0
renew_offer_type    0
sales_channel       0
vehicle_class       0
vehicle_size        0
dtype: int64

# 4. Check all unique values of columns.






In [24]:
for col in categorical_df.columns:
    print(col," : ",categorical_df[col].nunique(),categorical_df[col].unique(), )

state  :  5 ['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
response  :  2 ['No' 'Yes']
coverage  :  3 ['Basic' 'Extended' 'Premium']
education  :  5 ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
employmentstatus  :  5 ['Employed' 'Unemployed' 'Medical Leave' 'Disabled' 'Retired']
gender  :  2 ['F' 'M']
location_code  :  3 ['Suburban' 'Rural' 'Urban']
marital_status  :  3 ['Married' 'Single' 'Divorced']
policy_type  :  3 ['Corporate Auto' 'Personal Auto' 'Special Auto']
policy  :  9 ['Corporate L3' 'Personal L3' 'Corporate L2' 'Personal L1' 'Special L2'
 'Corporate L1' 'Personal L2' 'Special L1' 'Special L3']
renew_offer_type  :  4 ['Offer1' 'Offer3' 'Offer2' 'Offer4']
sales_channel  :  4 ['Agent' 'Call Center' 'Web' 'Branch']
vehicle_class  :  6 ['Two-Door Car' 'Four-Door Car' 'SUV' 'Luxury SUV' 'Sports Car'
 'Luxury Car']
vehicle_size  :  3 ['Medsize' 'Small' 'Large']


# 5. Check dtypes. Do they all make sense as categorical data?



For the ones that have 2 categories this is easy to replace by a number: 1 and 0

In [90]:
mapping = {'No':0,'Yes':1}
categorical_df = categorical_df.replace({'response': mapping})
mapping = {'M':0,'F':1}
categorical_df = categorical_df.replace({'gender': mapping})

# 6. Does any column contain alpha and numeric data? Decide how to clean it.


Column "renew_offer_type" has alfanumeric values, they can be replaced by a number:

In [91]:
mapping = {'Offer1':1,'Offer2':2,'Offer3':3,'Offer4':4}
categorical_df = categorical_df.replace({'renew_offer_type': mapping})

Policy type can be separated from policy because we have repeated data

In [92]:
categorical_df['policy_type'] = categorical_df.policy_type.apply(lambda x: x.split(" ")[0])
categorical_df['policy'] = categorical_df.policy.apply(lambda x: x[-1])

In [93]:
for col in categorical_df.columns:
    print(col," : ",categorical_df[col].nunique(),categorical_df[col].unique(), )

state  :  5 ['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
response  :  2 [0 1]
coverage  :  3 ['Basic' 'Extended' 'Premium']
education  :  4 ['Bachelor' 'College' 'High Education' 'High School or Below']
employmentstatus  :  3 ['Employed' 'Unemployed' 'other']
gender  :  2 [1 0]
location_code  :  3 ['Suburban' 'Rural' 'Urban']
marital_status  :  3 ['Married' 'Single' 'Divorced']
policy_type  :  3 ['Corporate' 'Personal' 'Special']
policy  :  3 ['3' '2' '1']
renew_offer_type  :  4 [1 3 2 4]
sales_channel  :  4 ['Agent' 'Call Center' 'Web' 'Branch']
vehicle_class  :  4 ['Two-Door Car' 'Four-Door Car' 'SUV' 'Luxury']
vehicle_size  :  3 [1 0 2]


# 7. Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions.


I could give ordinal values for some of the columns

In [98]:
mapping = {'Basic':0,'Extended':1,'Premium':2}
categorical_df = categorical_df.replace({'coverage': mapping})
mapping = {'Small':0,'Medsize':1,'Large':2}
categorical_df = categorical_df.replace({'vehicle_size': mapping})

In [113]:
# Split numerical and rest
categorical_df_num = categorical_df.drop(columns =['state','education','employmentstatus','location_code','policy_type','sales_channel','vehicle_class','marital_status'])
categorical_df_rest = categorical_df[['state','education','employmentstatus','location_code','policy_type','sales_channel','vehicle_class','marital_status']]
categorical_df_rest.head()

,state,education,employmentstatus,location_code,policy_type,sales_channel,vehicle_class,marital_status
0,Washington,Bachelor,Employed,Suburban,Corporate,Agent,Two-Door Car,Married
1,Arizona,Bachelor,Unemployed,Suburban,Personal,Agent,Four-Door Car,Single
2,Nevada,Bachelor,Employed,Suburban,Personal,Agent,Two-Door Car,Married
3,California,Bachelor,Unemployed,Suburban,Corporate,Call Center,SUV,Married
4,Washington,Bachelor,Employed,Rural,Personal,Agent,Four-Door Car,Single


# 8. Compare policy_type and policy. What information is contained in these columns. Can you identify what is important?


I did it in question 6

# 9. Check number of unique values in each column, can they be combined in any way to ease encoding? Comment your thoughts and make those changes.

In [115]:
for col in categorical_df_rest.columns:
    print(col,": \n",categorical_df_rest[col].value_counts(),"\n" )

state : 
 California    3150
Oregon        2601
Arizona       1703
Nevada         882
Washington     798
Name: state, dtype: int64 

education : 
 Bachelor                2748
College                 2681
High School or Below    2622
High Education          1083
Name: education, dtype: int64 

employmentstatus : 
 Employed      5698
Unemployed    2317
other         1119
Name: employmentstatus, dtype: int64 

location_code : 
 Suburban    5779
Rural       1773
Urban       1582
Name: location_code, dtype: int64 

policy_type : 
 Personal     6788
Corporate    1968
Special       378
Name: policy_type, dtype: int64 

sales_channel : 
 Agent          3477
Branch         2567
Call Center    1765
Web            1325
Name: sales_channel, dtype: int64 

vehicle_class : 
 Four-Door Car    4621
Two-Door Car     1886
SUV              1796
Luxury            831
Name: vehicle_class, dtype: int64 

marital_status : 
 Married     5298
Single      2467
Divorced    1369
Name: marital_status, dtype: int6

Maybe we can group vehicle class (Sports, Luxury SUV, Luxury car) into Luxury, also in education we can group (Doctor and Master) into high education, also employmentstatus ('Medical Leave' 'Disabled' 'Retired') into Other 

In [126]:
def Luxury_car(x):
    if ('Luxury' in x) or ('Sports' in x):
        x = "Luxury"
    return x

def High_education(x):
    if ('Master' in x) or ('Doctor' in x):
        x = "High Education"
    return x
    
def Employment(x):
    if ('Medical' in x) or ('Disabled' in x) or ('Retired' in x):
        x = "other"
    return x

categorical_df_rest2 = categorical_df_rest.copy()
categorical_df_rest2['vehicle_class'] = categorical_df_rest2['vehicle_class'].apply(Luxury_car)
categorical_df_rest2['education'] = categorical_df_rest2['education'].apply(High_education)
categorical_df_rest2['employmentstatus'] = categorical_df_rest2['employmentstatus'].apply(Employment)

In [127]:
for col in categorical_df_rest2.columns:
    print(col," : ",categorical_df_rest2[col].nunique(),categorical_df_rest2[col].unique(), )

state  :  5 ['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
education  :  4 ['Bachelor' 'College' 'High Education' 'High School or Below']
employmentstatus  :  3 ['Employed' 'Unemployed' 'other']
location_code  :  3 ['Suburban' 'Rural' 'Urban']
policy_type  :  3 ['Corporate' 'Personal' 'Special']
sales_channel  :  4 ['Agent' 'Call Center' 'Web' 'Branch']
vehicle_class  :  4 ['Two-Door Car' 'Four-Door Car' 'SUV' 'Luxury']
marital_status  :  3 ['Married' 'Single' 'Divorced']


We will apply OneHotEncoder on the rest:

In [138]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(categorical_df_rest2)
onehotlabels = enc.transform(categorical_df_rest2).toarray()
categorical_df_enc = pd.DataFrame(onehotlabels,columns = np.concatenate(enc.categories_))

In [147]:
categorical_total = pd.concat([categorical_df_num,categorical_df_enc], axis = 1)
categorical_total.head()

,response,coverage,gender,policy,renew_offer_type,vehicle_size,Arizona,California,Nevada,Oregon,...,Branch,Call Center,Web,Four-Door Car,Luxury,SUV,Two-Door Car,Divorced,Married,Single
0,0,0,1,3,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0,1,1,3,3,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,2,1,3,1,1,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0,0,0,2,1,1,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0,0,0,1,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
